## Consulta 03

*Esta consulta devuelve devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.*

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}



*Importar las librerías necesarias para el procesamiento de los datos*

In [1]:
import pandas as pd


*Importamos los archivos con los que vamos a trabajar la consulta*

In [2]:
df_games = pd.read_parquet(r'C:\Users\Owner\Desktop\LABS\Proyecto_STEAM\datos_STEAM\parquet\games_clean.parquet')
df_items = pd.read_parquet(r'C:\Users\Owner\Desktop\LABS\Proyecto_STEAM\datos_STEAM\parquet\items_clean.parquet')

*Realizamos una copia para trabajar sobre ella y mantener los datos intactos en el archivo original.*

In [3]:
df_games_copy = df_games.copy()
df_items_copy = df_items.copy()

*(Opcional). Podemos hacer una visualización previa del dataset para hacer cualquier análsis previo.*

In [4]:
df_games_copy

,id,app_name,developer,genres,price,release_date,tags
0,761140,Lost Summoner Kitty,Kotoshiro,Action,4.99,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]"
1,643980,Ironbound,Secret Level SRL,Indie,NaN,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game..."
2,670290,Real Pool 3D - Poolians,Poolians.com,Casual,NaN,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind..."
3,767400,弹炸人2222,彼岸领域,Action,0.99,2017-12-07,"[Action, Adventure, Casual]"
4,773570,Log Challenge,None,Action,2.99,None,"[Action, Indie, Casual, Sports]"
...,...,...,...,...,...,...,...
32130,773640,Colony On Mars,"Nikita ""Ghost_RUS""",Casual,1.99,2018-01-04,"[Strategy, Indie, Casual, Simulation]"
32131,733530,LOGistICAL: South Africa,Sacada,Casual,4.99,2018-01-04,"[Strategy, Indie, Casual]"
32132,610660,Russian Roads,Laush Dmitriy Sergeevich,Indie,1.99,2018-01-04,"[Indie, Simulation, Racing]"
32133,658870,EXIT 2 - Directions,"xropi,stev3ns",Casual,4.99,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe..."


In [5]:
df_items_copy

,user_id,items_count,item_id,playtime_forever
0,76561197970982479,277,10,6.0
1,76561197970982479,277,20,0.0
2,76561197970982479,277,30,7.0
3,76561197970982479,277,40,0.0
4,76561197970982479,277,50,0.0
...,...,...,...,...
5170010,76561198329548331,7,373330,0.0
5170011,76561198329548331,7,388490,3.0
5170012,76561198329548331,7,521570,4.0
5170013,76561198329548331,7,519140,3.0


*Realizamos las siguientes acciones para hacer la consulta que necesitamos:*
1. Verificar si la columna 'genres' está presente en el DataFrame df_games_copy.
2. Convertir la columna 'release_date' a tipo datetime.
3. Filtrar el DataFrame df_games_copy por el género proporcionado.
4. Unir el DataFrame filtrado con df_items_copy utilizando la columna 'id' en df_games_copy y 'item_id' en df_items_copy.
5. Calcular las horas jugadas por cada usuario para cada juego.
6. Encontrar el usuario que ha jugado más horas en total.
7. Calcular la acumulación de horas jugadas por año de lanzamiento para el género dado.
8. Retornar un diccionario con el usuario que ha jugado más horas para el género dado y la acumulación de horas jugadas por año de lanzamiento.

In [6]:
def UserForGenre(genero: str, df_games_copy, df_items_copy):
    # Verificar si la columna 'genre' está presente en el DataFrame df_games_copy
    if 'genres' not in df_games_copy.columns:
        raise ValueError("El DataFrame df_games_copy no tiene una columna llamada 'genre'.")

#Convertir la columna 'release_date' a tipo datetime
    df_games_copy['release_date'] = pd.to_datetime(df_games_copy['release_date'], errors='coerce')

#Filtrar df_games_copy por el género dado
    juegos_genero = df_games_copy[df_games_copy['genres'] == genero]

#Unir el DataFrame filtrado con df_user_items
    juegos_usuario = juegos_genero.merge(df_items_copy, left_on='id', right_on='item_id')

#Calcular las horas jugadas por usuario para cada juego
    horas_por_usuario = juegos_usuario.groupby('user_id')['playtime_forever'].sum().reset_index()

#Encontrar el usuario con más horas jugadas
    usuario_max_horas = horas_por_usuario.loc[horas_por_usuario['playtime_forever'].idxmax()]['user_id']

#Calcular la acumulación de horas jugadas por año de lanzamiento para el género dado
    horas_por_año = juegos_usuario.groupby(juegos_usuario['release_date'].dt.year)['playtime_forever'].sum().reset_index()
    horas_por_año.rename(columns={'playtime_forever': 'Horas'}, inplace=True)
    horas_por_año = horas_por_año.to_dict('records')

#Crear el diccionario de resultados
    result = {
        "Usuario con más horas jugadas para {}: ".format(genero): usuario_max_horas,
        "Horas jugadas": horas_por_año
    }

    return result

**Aplicamos la función y confirmamos que funcione correctamente*

In [9]:
genero = input("Ingrese el nombre del género: ")
resultado = UserForGenre(genero, df_games_copy, df_items_copy)
print(resultado)

{'Usuario con más horas jugadas para Action: ': 'Sp3ctre', 'Horas jugadas': [{'release_date': 1983.0, 'Horas': 3582.0}, {'release_date': 1984.0, 'Horas': 384.0}, {'release_date': 1988.0, 'Horas': 16243.0}, {'release_date': 1989.0, 'Horas': 607.0}, {'release_date': 1990.0, 'Horas': 18787.0}, {'release_date': 1991.0, 'Horas': 2502.0}, {'release_date': 1992.0, 'Horas': 1925.0}, {'release_date': 1993.0, 'Horas': 24663.0}, {'release_date': 1994.0, 'Horas': 121057.0}, {'release_date': 1995.0, 'Horas': 217813.0}, {'release_date': 1996.0, 'Horas': 70061.0}, {'release_date': 1997.0, 'Horas': 363033.0}, {'release_date': 1998.0, 'Horas': 2935053.0}, {'release_date': 1999.0, 'Horas': 2735338.0}, {'release_date': 2000.0, 'Horas': 18643971.0}, {'release_date': 2001.0, 'Horas': 1337303.0}, {'release_date': 2002.0, 'Horas': 740382.0}, {'release_date': 2003.0, 'Horas': 11969297.0}, {'release_date': 2004.0, 'Horas': 131222294.0}, {'release_date': 2005.0, 'Horas': 12426135.0}, {'release_date': 2006.0, 'H